In [1]:
import pandas as pd
import os
import matplotlib
import numpy as np
import json
import csv
import re

In [2]:
file_path="C:\\Users\shail\Movies-ETL\wikipedia-movies.json"

In [3]:
#Using with and load load the file as list of dictionaries
with open(file_path,mode='r') as file:
    wiki_movies_raw=json.load(file)
#print(wiki_movies_raw[0]['Produced by'])
len(wiki_movies_raw)

7311

In [4]:
len(wiki_movies_raw)
wiki_movies__raw_df=pd.DataFrame(wiki_movies_raw)
#wiki_movies_df.head()
wiki_movies_df.columns.tolist()

NameError: name 'wiki_movies_df' is not defined

In [ ]:
#loop through wiki_movies_raw and get only those movies who has a director and imdb link
wiki_movies=[movie for movie in wiki_movies_raw
              if ('Director' in movie or 'Directed by' in movie)
              and 'imdb_link' in movie
              and 'No. of episodes'not in movie
            ]
            

wiki_movies_df=pd.DataFrame(wiki_movies)
#wiki_movies_df.head(111)
#print(wiki_movies[111])

In [ ]:

wiki_movies_df[wiki_movies_df['Arabic'].notnull()]['url']

In [ ]:
def clean_movie(movie):
    movie = dict(movie)
    #List of alternative titles(languages)
    alternative_titles={}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alternative_titles[key]=movie[key]
            movie.pop(key)
    #print(movie['url'])
    #print(alternative_titles)
    if len(alternative_titles) >0:
        movie['alt_titles']= alternative_titles
        
    # merge column names
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
            #print(movie['url'])
            #print("............")
            #print(f"{new_name},{movie[new_name]}")
            
    change_column_name('Adaptation by', 'Writer(s)')
    change_column_name('Country of origin', 'Country')
    change_column_name('Directed by', 'Director')
    change_column_name('Distributed by', 'Distributor')
    change_column_name('Edited by', 'Editor(s)')
    change_column_name('Length', 'Running time')
    change_column_name('Original release', 'Release date')
    change_column_name('Music by', 'Composer(s)')
    change_column_name('Produced by', 'Producer(s)')
    change_column_name('Producer', 'Producer(s)')
    change_column_name('Productioncompanies ', 'Production company(s)')
    change_column_name('Productioncompany ', 'Production company(s)')
    change_column_name('Released', 'Release Date')
    change_column_name('Release Date', 'Release date')
    change_column_name('Screen story by', 'Writer(s)')
    change_column_name('Screenplay by', 'Writer(s)')
    change_column_name('Story by', 'Writer(s)')
    change_column_name('Theme music composer', 'Composer(s)')
    change_column_name('Written by', 'Writer(s)')
    
    
    
    
    return movie
    

In [ ]:
clean_movies=[clean_movie(movie) for movie in wiki_movies]
#print(clean_movies[0])

In [ ]:
wiki_movies_df = pd.DataFrame(clean_movies)
sorted(wiki_movies_df.columns.tolist())


In [ ]:
#Columns are cleaned up..now clean up the rows
#Extract imdb id from movie url
wiki_movies_df['imdb_id']=wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
print(len(wiki_movies_df))
wiki_movies_df.drop_duplicates(subset='imdb_id',inplace=True)
print(len(wiki_movies_df))

In [ ]:
wiki_movies_df.columns

In [ ]:
#Drop all the null values from all the columns
wiki_columns_to_keep = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]
wiki_movies_df = wiki_movies_df[wiki_columns_to_keep]
wiki_movies_df

In [ ]:
len(wiki_movies_df)

In [ ]:
#Alternative for loop
for column in wiki_movies_df.columns:
    col=wiki_movies_df[column].isnull().sum()
    print(f"{column} : {col}")

In [ ]:
wiki_movies_df.dtypes

In [ ]:
print(len(wiki_movies_df['Box office']))
box_office = wiki_movies_df['Box office'].dropna()
len(box_office)
wiki_movies_df['Box office'].head(60)

In [ ]:
def is_not_a_string(x):
    return type(x) != str

box_office[box_office.map(is_not_a_string)]

In [ ]:
box_office.map(lambda x: type(x) != str)

In [ ]:
#lambda version of is not a string function
#lambda x: type(x) != str

box_office[box_office.map(lambda x: type(x) != str)]

In [ ]:
#few of the values are stored in list.. we can join the values into strings
box_office= box_office.apply(lambda x: ' '.join(x) if type(x) == list else x)
box_office.head(45)

In [ ]:
#use regex to find the amount type is in which form
# form 1 is of the type $123.4 million/billion.

form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
box_office.str.contains(form_one, flags=re.IGNORECASE).sum()


In [ ]:
#form 2 is of the type "$123,456,789"

form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
box_office.str.contains(form_two, flags=re.IGNORECASE).sum()

In [ ]:
matches_form_one = box_office.str.contains(form_one, flags=re.IGNORECASE)
matches_form_two = box_office.str.contains(form_two, flags=re.IGNORECASE)

In [ ]:
#find values which are neither type 1 or 2
box_office[~matches_form_one & ~matches_form_two].count()

In [ ]:
#make a function which takes in a string
# matches that against the regex provided
def parse_dollars(s):
    #if s is not a string return nan
    if type(s) != str:
        return np.nan
    
    #if input is of the form $###.# million  $566.6 milliion
    if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
        #remove dollar signs, spaces, commas, and letters
        s = re.sub('\$|\s|[a-zA-Z]','', s)
        
        # convert to float and multiply by a million
        value = float(s) * 10**6

        # return value
        return value
    
    #if input if of the form ####.# billion
    elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
        #remove dollar sign, spaces, commas and letters
        s=re.sub('\$|\s|[a-zA-Z]','',s)
        
        #convert to float and multiple by billion
        value=float(s) * 10**9
        return value
    
    #if input is of the form $###,###,###
    elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)',s,flags=re.IGNORECASE):
        #remove $ sign and comma
        s = re.sub('\$|,','', s)
        # convert to float
        value = float(s)
        # return value
        return value
    
    else:
        return np.nan
    

In [ ]:
#wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [ ]:
#load the kaggle data into df
movies_metadata="C:\\Users\shail\Movies-ETL\movies_metadata.csv"
movies_metadata_pd=pd.read_csv(movies_metadata,low_memory="False")

#movies_metadata_pd.head()

In [ ]:
wiki_movies_df['box_office'] = box_office.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [ ]:
print(wiki_movies_df['box_office'])

In [ ]:
wiki_movies_df.drop('Box office', axis=1, inplace=True)

In [ ]:
ratings_path=os.path.join('ratings.csv')
ratings_df=pd.read_csv(ratings_path)
ratings_df.sample(n=7)

In [ ]:
wiki_movies_df.columns

In [ ]:
#Now clean the budget colum the same way as box_office columm
wiki_movies_df['Budget'].count()
budget=wiki_movies_df['Budget'].dropna()


In [ ]:
budget

In [ ]:
budget.tail()

In [ ]:
#Convert any lists to strings:
budget.map(lambda x:' '.join(x) if type(x) == list else x)

In [ ]:
budget.to_csv('export_budget.csv')

In [ ]:
#Then remove any values between a dollar sign and a hyphen (for budgets given in ranges):  $20-25 million or $13,000,000 (Estimated)
budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)    

In [ ]:
budget

In [ ]:
budget = budget.str.replace(r'\[\d+\]\s*', '')

In [ ]:

matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na= False)


#box_office[~matches_form_one & ~matches_form_two].count()
budget[~matches_form_one & matches_form_two]
#matches_form_one

In [ ]:
wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

In [ ]:
wiki_movies_df.drop('Budget', axis=1, inplace=True)

In [ ]:
#parse the release date column now
wiki_movies_df.columns

In [ ]:
wiki_movies_df['Release date']

In [ ]:
#converting all values from list to string
release_date=wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x)== list else x)

In [ ]:
#Parsing Full month name, one- to two-digit day, four-digit year (i.e., January 1, 2000)
date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'

#Four-digit year, two-digit month, two-digit day, with any separator (i.e., 2000-01-01)
date_form_two=r'\d{4}.[01]\d.[123]\d'

#Full month name, four-digit year (i.e., January 2000)
date_form_three=r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'

#Four-digit year
date_form_four=r'\d{4}'

In [ ]:
release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

In [ ]:
wiki_movies_df['release_date'] = pd.to_datetime(release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)

In [ ]:
release_date.isnull().count()

In [ ]:
#Parse running type
running_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)

In [ ]:
running_time

In [ ]:
#Use string boundaries to find how many values are of the type "100 minutes etc"

print(running_time[running_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE) != True])

In [ ]:
#filteringm more by accepting other abbreviations of minutes by putting only m
running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE).sum()

In [ ]:
running_time[running_time.str.contains(r'\d*\s*m', flags=re.IGNORECASE) != True]

In [ ]:
running_time_extract = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')

In [ ]:
running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)

In [ ]:
wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)

In [ ]:
wiki_movies_df.drop('Running time', axis=1, inplace=True)

In [15]:
#load the kaggle data into df
movies_metadata="C:\\Users\shail\Movies-ETL\movies_metadata.csv"
kaggle_metadata=pd.read_csv(movies_metadata,low_memory="False")

#movies_metadata_pd.head()
kaggle_metadata.dtypes

C:\Users\shail\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3166: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object

In [16]:
kaggle_metadata['adult'].value_counts()

False                                                                                                                             45454
True                                                                                                                                  9
 - Written by Ørnås                                                                                                                   1
 Rune Balot goes to a casino connected to the October corporation to try to wrap up her case once and for all.                        1
 Avalanche Sharks tells the story of a bikini contest that turns into a horrifying affair when it is hit by a shark avalanche.        1
Name: adult, dtype: int64

In [17]:
#Remove the bad data from above cell
kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
19730,- Written by Ørnås,0.065736,/ff9qCepilowshEtG2GYWwzt2bs4.jpg,"[{'name': 'Carousel Productions', 'id': 11176}...","[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso...",1997-08-20,0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29503,Rune Balot goes to a casino connected to the ...,1.931659,/zV8bHuSL6WXoD6FWogP9j4x80bL.jpg,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...","[{'iso_3166_1': 'US', 'name': 'United States o...",2012-09-29,0,68.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,...,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35587,Avalanche Sharks tells the story of a bikini ...,2.185485,/zaSf5OG7V8X8gqFvly88zDdRm46.jpg,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...","[{'iso_3166_1': 'CA', 'name': 'Canada'}]",2014-01-01,0,82.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#fix the jumbled columns from above cell
kaggle_metadata = kaggle_metadata[kaggle_metadata['adult'] == 'False'].drop('adult',axis='columns')
print(kaggle_metadata)

                                   belongs_to_collection    budget  \
0      {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1                                                    NaN  65000000   
2      {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3                                                    NaN  16000000   
4      {'id': 96871, 'name': 'Father of the Bride Col...         0   
...                                                  ...       ...   
45461                                                NaN         0   
45462                                                NaN         0   
45463                                                NaN         0   
45464                                                NaN         0   
45465                                                NaN         0   

                                                  genres  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1      [{'id': 12, 'name': 'Adventure'}

In [21]:
#Parse the video column 
kaggle_metadata['video'].value_counts()

False    45358
True        93
Name: video, dtype: int64

In [23]:
#boolean column with true values
kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'